## Import / Config

In [9]:
import pandas as pd
from sqlalchemy import create_engine, text
from neo4j import GraphDatabase
import cred_pg as c
import cred_neo4j as cc

## Engine

In [10]:
# Postgres Driver erstellen
engine_postgres = create_engine(
    f'postgresql+psycopg://{c.pg_userid}:{c.pg_password}@{c.pg_host}/{c.pg_db}',
    connect_args={'options': '-c search_path=$user,ugeobln,umisc,umobility,usozmed,public', 'keepalives_idle': 120},
    pool_size=1,
    max_overflow=0,
    execution_options={'isolation_level': 'AUTOCOMMIT'}
)

# Neo4j Driver erstellen
engine_neo4j = GraphDatabase.driver(cc.neo4j_host, auth=(cc.neo4j_userid, cc.neo4j_password))


In [11]:
# engine.dispose()

## Code

In [12]:
def get_closest_station(lat_lng):
    # Erhalte am naechsten gelegene Station, basieren auf Latitude und Longitude
    with engine_postgres.connect() as con:
        query = f"""
            SELECT hid, bez,
                ST_Distance(
                pos,
                'SRID=4326;POINT({lat_lng[1]} {lat_lng[0]})'::geography
            ) AS distanz_in_meter
        FROM haltestelle
        ORDER BY distanz_in_meter ASC
        LIMIT 1;
        """
        df = pd.read_sql_query(text(query), con)
        return df


def get_path_start_finish(station_start, station_ziel):
    # Erhalte Pfad der zu fahren ist, basierend auf Start und Endhaltestelle
    with engine_neo4j.session() as session:
        query = f"""
            MATCH (h1:Haltestelle {{bez: '{station_start.iloc[0]["bez"]}'}}), (h2:Haltestelle {{bez: '{station_ziel.iloc[0]["bez"]}'}})
            CALL gds.shortestPath.dijkstra.stream('bubahn', {{
                sourceNode: h1,
                targetNode: h2
                }})
            YIELD index, sourceNode, targetNode, nodeIds
            with [nodeId IN nodeIds | gds.util.asNode(nodeId)] AS p
            UNWIND p as station
            RETURN station.bez as bez, station.hid as hid;
        """
        res = session.run(query)
        return pd.DataFrame(res.data())


def get_seg_between_stations(station_a_id, station_b_id):
    # Erhalte Segment zwischen zwei Stationen
    with engine_postgres.connect() as con:
        query = f"""
            SELECT uu.ulid, ul.bez
            FROM umobility.abschnitt AS ua
                INNER JOIN umobility.unterlinie AS uu ON ua.ulid = uu.ulid
                INNER JOIN umobility.linie AS ul ON uu.lid = ul.lid
            WHERE hid_a = '{station_a_id}'
                AND hid_b = '{station_b_id}';
        """
        df = pd.read_sql_query(text(query), con)
        return df


def get_departure_time(station_id, time, ulid, date):
    # Erhalte Abfahrstzeit einer Station, zu einem Zeitpunkt, an einem Datum, mit einer bestimmten ULID
    with engine_postgres.connect() as con:
        query = f"""
            SELECT halt.zeit_abfahrt as zeit
            FROM umobility.zeitplan AS zeitplan
                INNER JOIN umobility.fahrt AS fahrt ON zeitplan.zpid = fahrt.zpid
                INNER JOIN umobility.halt AS halt ON fahrt.fid = halt.fid
            WHERE ('{date}' BETWEEN zeitplan.datum_beginn AND zeitplan.datum_ende)
                AND halt.hid = '{station_id}'
                AND halt.zeit_abfahrt > '{time}'
                AND fahrt.ulid = {ulid}
            ORDER BY halt.zeit_abfahrt ASC
            LIMIT 1;
        """
        df = pd.read_sql_query(text(query), con)
        return df.iloc[0]["zeit"]


def get_arrival_time(station_id, time, ulid, date):
    # Erhalte Startzeit einer Station, zu einem Zeitpunkt, an einem Datum, mit einer bestimmten ULID
    with engine_postgres.connect() as con:
        query = f"""
            SELECT halt.zeit_ankunft as zeit
            FROM umobility.zeitplan AS zeitplan
                INNER JOIN umobility.fahrt AS fahrt ON zeitplan.zpid = fahrt.zpid
                INNER JOIN umobility.halt AS halt ON fahrt.fid = halt.fid
            WHERE ('{date}' BETWEEN zeitplan.datum_beginn AND zeitplan.datum_ende)
                AND halt.hid = '{station_id}'
                AND halt.zeit_ankunft > '{time}'
                AND fahrt.ulid = {ulid}
            ORDER BY halt.zeit_ankunft ASC
            LIMIT 1;
        """
        df = pd.read_sql_query(text(query), con)
        return df.iloc[0]["zeit"]


def fahrverbindung(lat_lng_start, lat_lng_ziel, startdatum, startzeit):
    # Erhalte nachste Startstation
    station_start = get_closest_station(lat_lng_start)
    print(f"Starthaltestelle\n{station_start}")

    # Erhalte naechste Zielstation
    station_ziel = get_closest_station(lat_lng_ziel)
    print(f"\nZielhaltestelle\n{station_ziel}")

    # Erhalte Pfad der gefahren werden muss
    pfad = get_path_start_finish(station_start, station_ziel)

    results = []

    line_prev = ""
    time_current = ""

    # Iteriere über Eintraege im Pfad und erstelle dict eintrag in results liste
    for i in range(len(pfad) - 1):
        line_info = get_seg_between_stations(pfad.iloc[i]["hid"], pfad.iloc[i + 1]["hid"])
        line = line_info.iloc[0]["bez"]
        line_ulid = line_info.iloc[0]["ulid"]

        # Ueberpruefe ob Linie gewechselt wird
        if line_prev != "":
            if line_prev == line:
                umstieg = "nein"
            else:
                umstieg = "ja"
                line_prev = line
        else:
            line_prev = line
            umstieg = "nein"

        # Aktualisiere Zeit basierend auf fortschritt
        if time_current == "":
            time_current = startzeit

        time_departure = get_departure_time(pfad.iloc[i]["hid"], time_current, line_ulid, startdatum)
        time_current = time_departure
        time_arrival = get_arrival_time(pfad.iloc[i + 1]["hid"], time_current, line_ulid, startdatum)
        time_current = time_arrival

        # Erstelle Eintrag fuer Liste
        table_entry = {
            "hid1": f"{pfad.iloc[i]["hid"]}",
            "hid2": f"{pfad.iloc[i + 1]["hid"]}",
            "von": f"{pfad.iloc[i]["bez"]}",
            "nach": f"{pfad.iloc[i + 1]["bez"]}",
            "U": f"{line}",
            "Abfahrt": f"{time_departure}",
            "Ankunft": f"{time_arrival}",
            "Umstieg": f"{umstieg}",
        }

        results.append(table_entry)

    df_result = pd.DataFrame(results)
    print(f"\nVerbindung\n{df_result}")

    return

## Ausführung

### Eberbacher Str. 1 nach Meierottostraße 10

In [13]:
fahrverbindung([52.473176, 13.313740], [52.4969134, 13.327166], "2021-04-01", "11:00:00")

Starthaltestelle
     hid               bez  distanz_in_meter
0  10291  RüdesheimerPlatz         90.804351

Zielhaltestelle
     hid          bez  distanz_in_meter
0  10303  SpichernStr        236.990577

Verbindung
    hid1   hid2                von               nach   U   Abfahrt   Ankunft  \
0  10291  10152   RüdesheimerPlatz  HeidelbergerPlatz  U3  11:00:46  11:01:39   
1  10152  10209  HeidelbergerPlatz  FehrbellinerPlatz  U3  11:02:09  11:03:30   
2  10209  10229  FehrbellinerPlatz  Hohenzollernplatz  U3  11:04:00  11:04:51   
3  10229  10303  Hohenzollernplatz        SpichernStr  U3  11:05:21  11:05:57   

  Umstieg  
0    nein  
1    nein  
2    nein  
3    nein  


### Eberbacher Str. 1 nach Ballenstedter Str. 6

In [14]:
fahrverbindung([52.473176, 13.313740], [52.494833, 13.3038814], "2021-04-01", "11:00:00")

Starthaltestelle
     hid               bez  distanz_in_meter
0  10291  RüdesheimerPlatz         90.804351

Zielhaltestelle
     hid            bez  distanz_in_meter
0  10242  KonstanzerStr         444.20619

Verbindung
    hid1   hid2                von               nach   U   Abfahrt   Ankunft  \
0  10291  10152   RüdesheimerPlatz  HeidelbergerPlatz  U3  11:00:46  11:01:39   
1  10152  10209  HeidelbergerPlatz  FehrbellinerPlatz  U3  11:02:09  11:03:30   
2  10209  10242  FehrbellinerPlatz      KonstanzerStr  U7  11:04:29  11:05:01   

  Umstieg  
0    nein  
1    nein  
2      ja  


### Eberbacher Str. 1 nach Elsastraße 2

In [15]:
fahrverbindung([52.473176, 13.313740], [52.4752505, 13.3308319], "2021-04-01", "11:00:00")

Starthaltestelle
     hid               bez  distanz_in_meter
0  10291  RüdesheimerPlatz         90.804351

Zielhaltestelle
     hid          bez  distanz_in_meter
0  10148  Bundesplatz        261.707791

Verbindung
    hid1   hid2                von               nach   U   Abfahrt   Ankunft  \
0  10291  10152   RüdesheimerPlatz  HeidelbergerPlatz  U3  11:00:46  11:01:39   
1  10152  10209  HeidelbergerPlatz  FehrbellinerPlatz  U3  11:02:09  11:03:30   
2  10209  10195  FehrbellinerPlatz          BlisseStr  U7  11:05:31  11:06:10   
3  10195  10189          BlisseStr        BerlinerStr  U7  11:06:40  11:07:29   
4  10189  10148        BerlinerStr        Bundesplatz  U9  11:09:45  11:10:57   

  Umstieg  
0    nein  
1    nein  
2      ja  
3    nein  
4      ja  
